In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import xarray as xr
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import QuantileTransformer
from xarray import DataArray, Dataset

from pipeline.backtest import cross_validation
from visualization.metric import plot_performance

base_ds = xr.open_dataset('../../data/nc/base.nc')
market_brief_ds = xr.open_dataset('../../data/nc/market_brief.nc')
fundamental_v0_ds = xr.open_dataset('../../data/nc/fundamental_v0.nc')
ds = base_ds.merge(market_brief_ds).merge(fundamental_v0_ds)

feature = ['book', 'cashflow', 'sales', 'earnings_ttm', 'earnings', 'market_cap']

In [16]:
from pandas import Series


class SimpleLinearModel:
    def __init__(self, feature):
        self.feature = feature
        self.reg = LinearRegression()
        self.scaler = QuantileTransformer(n_quantiles=10)

    def fit_predict(self, X: Dataset, y: DataArray) -> DataArray:
        dim_order = 'day', 'asset'
        X_pd = X[self.feature].to_dataframe(dim_order=dim_order)
        X_transformed_np = self.scaler.fit_transform(X_pd.values)
        y_np = y.stack({'stack': ['day', 'asset']}).as_numpy()
        self.reg.fit(X_transformed_np, y_np)
        y_pred_np = self.reg.predict(X_transformed_np)
        return DataArray.from_series(Series(y_pred_np, index=X_pd.index))

    def predict(self, X: Dataset) -> DataArray:
        dim_order = 'day', 'asset'
        X_pd = X[self.feature].to_dataframe(dim_order=dim_order)
        X_transformed_np = self.scaler.transform(X_pd.values)
        y_pred_np = self.reg.predict(X_transformed_np)
        return DataArray.from_series(Series(y_pred_np, index=X_pd.index))

In [17]:
model = SimpleLinearModel(feature)
train_lookback = 30
performance_cv, cum_y_df = cross_validation(model, feature, ds, train_lookback=train_lookback)

  0%|          | 0/969 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [5]:
plt.figure(1)
plot_performance(performance_cv, metrics_selected=['train_r2', 'val_cum_r2', 'val_cum_pearson'])
plt.show()

NameError: name 'performance_cv' is not defined

<Figure size 640x480 with 0 Axes>

In [18]:
from qids_lib import QIDS
qids = QIDS(path_prefix='../../')

In [ ]:
from pipeline.backtest import evaluation_for_submission

model = SimpleLinearModel(feature)
train_lookback = 3
performance_eval = evaluation_for_submission(model, ds, qids, lookback_window=train_lookback)

  0%|          | 0/700 [00:00<?, ?it/s]